## BERT Model Outcomes from 1960-1969

In [1]:
import pandas as pd
import numpy as np 

In [ ]:
# importing cleaned and processed data
df = pd.read_csv('../../data/clean_processed.csv')

In [ ]:
# Filtering data for only the 1960s
df = df[df['Year'].between(1960, 1969)]
df['Year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969])

In [ ]:
# Checking dimensions of filtered data
df.shape

(942, 11)

In [5]:
df.head(1)

,Year,Artist,Song Title,Rank,Lyrics,Nouns,Verbs,Adverbs,Corpus,Word Counts,Unique Word Counts
87,1960,Percy Faith,"Theme From ""A Summer Place""",1,There's a summer place Where it may rain or st...,summer place storm summer place arm heart care...,be rain reach know be see bless share share,anywhere anywhere,summer place rain storm safe warm summer place...,104,64


In [ ]:
import re
from string import punctuation

# Light cleaning function to clean lyrics
def light_clean(text):
    if pd.isna(text):
        return ""
    # converting to lowercase
    text = text.lower()
    # removing special characters
    text = ''.join([char for char in text if char not in punctuation])
    # removing numbers 
    text = re.sub(pattern = r'\d+',
                  repl = '',
                   string = text)
    # removing extra whitespace
    text = re.sub(pattern = r'\s+',
                  repl = ' ',
                  string = text)
    
    cleaned = text  
    return cleaned

df['clean_lyrics'] = df['Lyrics'].apply(light_clean)

In [ ]:
# creating list of cleaned lyrics
lyrics = df['clean_lyrics'].tolist()

## Build and Fit BERTopic Model

In [ ]:
# Importing necessary libraries for BERT
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [ ]:
# Creating custom stopword list for song lyrics
stopwords_nltk = stopwords.words('english')
lyrics_stopwords = stopwords_nltk + [
    'la', 'lo', 'lola', 'du', 'oh', 'yeah', 'la la', 'lo lo', 'du du', 'got', 'gonna',
    'ooh', 'get', 'take', 'da', 'know', 'like', 'one', 'said', 'ron',
    'let', 'go', 'come', 'want', 'toke', 'say', 'day', 'good', 'see', 'make',
    'niggas', 'nigga', 'ya', 'choo', 'na', 'uh', 'yuh', 'hmm', 'wanna',
    'doo', 'dat', 'woo', 'nah', 'whoomp', 'yo', 'whoo', 'whatta', 'wa',
    'johnny', 'boaw', 'th', 'whoa', 'eh', 'ooo', 'um', 'dum', 'thy', 'aa',
    'the', 'on', 'and', 'ta', 'pa', 'oop', 'ha', 'bop', 'ah', 'yah', 'shes',
    'oo', 'im', 'ohoh', 'whats', 'huh', 'youre','dont', 'self', 'isnt', 'yeh', 
    'jo', 'cant', 'hes', 'uhuh', 'ive', 'yah', 'hmmmmm', 'ohh', 'woulda', 'nae',
    'lil', 'chh', 'ayy', 'aha', 'dit', 'ding', 'ling', 'feat', 'justin', 'mbabarara',
    'rainin', 'burnin', 'em', 'drake', 'sylvia', 'alejandro', 'mmm', 'watcha'
    'somethin', 'rumours', 'girl', 'lovin', 'baby', 'lil', 'cara', 'ill', 'ba',
    'monday', 'sunday', 'yes', 'things','timin','mia'
    
]

vectorizer = CountVectorizer(stop_words=lyrics_stopwords)

In [ ]:
# More libraries for topic modeling 
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN

In [ ]:
# Dimensionality reduction
umap_model = UMAP(
    n_neighbors=10,       # local vs global structure
    n_components=10,      # dimensionality of reduced space
    min_dist=0.0,         # tighter clusters
    metric="cosine",
    random_state=42)

# Density-based clustering
hdbscan_model = HDBSCAN(
    min_cluster_size=20,  # how many docs per topic (granularity)
    min_samples=10,        # how strict the clustering is
    prediction_data=True,
    cluster_selection_method="leaf") # how clusters are calculated -> "leaf" results in more granular clusters 

# Topic model
topic_model_v3 = BERTopic(
    embedding_model= 'all-distilroberta-v1',  
    vectorizer_model=vectorizer,          # custom stopwords here
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    min_topic_size=20,                    # smallest amount of document allowed in a topic -> smaller number means more granular topics
    calculate_probabilities=True,
    verbose=True)

topics_v3, probabilities_v3 = topic_model_v3.fit_transform(lyrics)

2025-11-27 23:40:22,503 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

2025-11-27 23:45:46,629 - BERTopic - Embedding - Completed ✓
2025-11-27 23:45:46,630 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-27 23:45:50,119 - BERTopic - Dimensionality - Completed ✓
2025-11-27 23:45:50,120 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-27 23:45:50,197 - BERTopic - Cluster - Completed ✓
2025-11-27 23:45:50,201 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-27 23:45:50,399 - BERTopic - Representation - Completed ✓


In [26]:
topic_model_v3.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,518,-1_love_little_never_heart,"[love, little, never, heart, hey, time, well, ...",[ you see this guy this guys in love with you ...
1,0,93,0_proof_funny_mow_bird,"[proof, funny, mow, bird, home, well, back, ba...",[a well a everybodys heard about the bird bbb ...
2,1,67,1_love_true_kiss_angel,"[love, true, kiss, angel, hold, mine, way, nev...",[i love how your eyes close whenever you kiss ...
3,2,65,2_love_time_tell_never,"[love, time, tell, never, fool, mind, pieces, ...",[ youre a no good heartbreaker youre a liar an...
4,3,56,3_love_need_way_sweet,"[love, need, way, sweet, real, please, mine, j...",[have i ever told you how good it feels to hol...
5,4,45,4_twist_shake_boogety_dance,"[twist, shake, boogety, dance, lets, shoobop, ...",[twist twist senora hey now hey now twist all ...
6,5,40,5_summer_away_stand_home,"[summer, away, stand, home, pleasant, moon, va...",[ill be alone each and every night while youre...
7,6,35,6_cry_lonely_gone_since,"[cry, lonely, gone, since, love, ronnie, talk,...",[lover youve gone from me and left behind so m...
8,7,23,7_happy_time_washed_think,"[happy, time, washed, think, free, land, id, j...",[ooh ooh ooh ooh ooh ooh ooh if i had a hammer...


### Review Interpretable Topics

In [ ]:
# Details of love 
# Singer's desire for their lover to show their love 

lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[1]

# Topic Theme = Affections of Love 

['i love how your eyes close whenever you kiss me and when im away from you i love how you miss me i love the way you always treat me tenderly but darling most of all i love how you love me i love how your heart beats whenever i hold you i love how you think of me without being told to i love the way your touch is always heavenly but darling most of all i love how you love me i love how your eyes close whenever you kiss me and when im away from you i love how you miss me i love the way your touch is always heavenly but darling most of all i love how you love me i love how you hug me i love how you squeeze me tease me please me love how you love me i love how you love me',
 'ill never let you go why because i love you ill always love you so why because you lome me no broken hearts for us cause we love each other and with our faith and trust there could be no other why cause i love you why cause you love me i think youre awfully sweet why because i love you you say im your special treat 

In [ ]:
# Pain and Frustration from a tumultuous relationship
# External pressure on a relationship; people trying to break them up
# self-destructive nature of love 

lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[2]

# Topic Theme = Turmoil in Relationships

[' youre a no good heartbreaker youre a liar and youre a cheat and i dont know why i let you do these things to me my friends keep telling me that you aint no good but oh they dont know that id leave you if i could i guess im uptight and im stuck like glue cause i aint never i aint never i aint never no no loved a man the way that i i love you some time ago i thought you had run out of fools but i was so wrong you got one that youll never lose the way you treat me is a shame how could you hurt me so bad baby you know that im the best thing that you ever had kiss me once again dont you never never say that we were through cause i aint never never never no no loved a man the way that i i love you i cant sleep at night and i cant eat a bite i guess ill never be free since you got your hooks in me oh oh oh yeah yeah i aint never loved a man i aint never loved a man baby aint never had a man hurt me so bad no well this is what im going to do about it i tell you im gonna',
 'whoa cant you se

In [ ]:
# Song tied to a new dance style "The Twist"
# song about dancing and having fun 

lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[4]

# Topic Theme = Twist Dance Craze

['twist twist senora hey now hey now twist all around what you say now twist twist senora come on come on shake it up and down one more time now jump jump senora let me hear you now jump all around come on come on twist twist senora well all right twist it all around well theres a real swinging dance called the twist now come on and get with it young and old dance together come on and dance bring your partner twist twist senora come on come on twist all around well well alright twist twist senora yeah oh yeah shake it up and down let me hear you now jump jump senora jump up in the air now jump through the night let me hear you now twist twist senora come on come on twist it all around well when i say stick your shoulders i mean so left or right to the temple and in case you get a notion jump in the air come down in slow motion twist twist senora come on come on twist all around twist with me now twist twist senora come on come on shake it up and down come on baby jump jump senora let m

In [ ]:
# Bittersweet parting of a summer romance 
# Love song of people parting for the summer 

lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[5]

# Topic Theme = Summer Romances/ Bittersweet Partings

['ill be alone each and every night while youre away dont forget to write byebye so long farewell byebye so long see you in september see you when the summers through here we are bye baby goodbye saying goodbye at the station bye baby goodbye summer vacation bye baby bye baby is taking you away bye baby goodbye have a good time but remember there is danger in the summer moon above will i see you in september or lose you to a summer love counting the days till ill be with you counting the hours and the minutes too bye baby goodbye bye baby goodbye bye baby goodbye byebye so long farewell bye baby goodbye byebye so long have a good time but remember there is danger in the summer moon above will i see you in september or lose you to a summer love ill be alone each and every night while youre away dont forget to write see you byebye so long farewell in september byebye so long farewell im hopin ill see you byebye so long farewell in september byebye so long farewell well maybe ill see you 

In [ ]:
# Painful feeling of loss; breakup or death 
# Pain from a breakup; seeking revenge 
# Emptiness felt after breakup 

lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[6]

# Topic Theme = Heartbreak 

['lover youve gone from me and left behind so many memories heres your old friendship ring i cant wear it no more heres your old love letters i cant read them any more lover youve gone from me and left behind so many memories heres that old teddy bear come and get it that you won for me at the state fair heres some old valentine cards come and get em give them to your new sweetheart lover youve gone from me and left behind so many memories yeah yeah yeah yeah come and get em come and get em come and get em heres our old favorite record come and get it i cant stand to hear it anymore heres some old lingering love come and get it its in my heart and its tearing it apart because of these memories i never think of anybody but you so come on and get em cause i found me somebody new come and get these memories come and get em since youve gone out of my life so my mind and my heart can leave that easy come and get em since youve gone out of my life give them to your new love give them to your

### Adding Topics to Original DataFrame 

In [37]:
df['Topics'] = topics_v3
df.head()

,Year,Artist,Song Title,Rank,Lyrics,Nouns,Verbs,Adverbs,Corpus,Word Counts,Unique Word Counts,clean_lyrics,Topics
87,1960,Percy Faith,"Theme From ""A Summer Place""",1,There's a summer place Where it may rain or st...,summer place storm summer place arm heart care...,be rain reach know be see bless share share,anywhere anywhere,summer place rain storm safe warm summer place...,104,64,theres a summer place where it may rain or sto...,5
88,1960,Jim Reeves,He'll Have To Go,2,Put your sweet lips a little closer to the ph...,lip phone man friend way love mind word man li...,put let pretend tell turn tell have go tell lo...,little close together all alone way down low t...,sweet lip little close phone let pretend tell ...,152,76,put your sweet lips a little closer to the ph...,2
89,1960,Everly Brothers,Cathy's Clown,3,Don't want your love any more Don't want your...,love kiss time sound clown man lie man tear lo...,want want die hear come get stand know crawl k...,more here at all kind of so even more here,want love want kiss sure die time hear sound c...,122,71,dont want your love any more dont want your k...,-1
90,1960,Johnny Preston,Running Bear,4,"On the bank of the river Stood Running Bear, y...",bank river side river maid name sight tribe lo...,stand see fight run love run love die swim rag...,never too fast there now always together,bank river stood running bear young indian bra...,220,100,on the bank of the river stood running bear yo...,-1
91,1960,Mark Dinning,Teen Angel,5,Teen Angel Teen Angel Teen Angel VERSE 1 That...,night car railroad track angel angel love vers...,stall pull go run hear see look take say find ...,back somewhere up above still just now away ne...,teen angel teen angel teen angel verse fateful...,118,83,teen angel teen angel teen angel verse that fa...,-1


In [ ]:
#df.to_csv('bert1960s.csv', index=False)